## Deploy Monitor

- get monitor project for arm(rapsberry pi) - https://github.com/6za/node-exporter-collectors
- deploy on all nodes - `docker-compose`
- `curl` to check connection

### Get Project 

Get [6za/node-exporter-collectors](https://github.com/6za/node-exporter-collectors) to deploy on nodes.


In [112]:
%%bash

mkdir ~/repos
cd ~/repos
#rm -rf ~/repos/node-exporter-collectors
git clone https://github.com/6za/node-exporter-collectors.git

mkdir: cannot create directory '/root/repos': File exists
Cloning into 'node-exporter-collectors'...


### Deploying on ARM

On Arm, there is extra monitor as part of the image to collect temperature from some sensors.


Details at: [6za/prometheus-raspberry-exporter](https://github.com/6za/prometheus-raspberry-exporter)

The images in use on `raspberry pi` are done using: [6za/pi-gen-vagrant](https://github.com/6za/pi-gen-vagrant)

And customized with: [6za/pi-gen](https://github.com/6za/pi-gen)


We will:
 - List from all hosts the ones based on `raspberry pi` and arch `armv7l`
 - deploy the arm version of the `node-exporter`
 - check the deployed containers

In [140]:
import pandas as pd
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']
select_hosts = hosts[(hosts.user == "pi") & (hosts.arch == "armv7l")]
select_hosts['ip'].to_csv(r'arm-ip.txt', header=False, index=None, sep=' ')
select_hosts.drop(columns=supressed_columns)
hosts['ip'].to_csv(r'ip.txt', header=False, index=None, sep=' ')

In [141]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/node-exporter-collectors
    export DOCKER_HOST="tcp://$ip:2376"
    docker-compose -f docker-compose-raspi3.yaml down 2>/dev/null
    docker-compose -f docker-compose-raspi3.yaml up -d 2>/dev/null
done <arm-ip.txt
cd $HOME_DIR

/root/04-deploy-monitor


### Deploy x86



In [142]:
hosts[(hosts.arch == "x86_64")]['ip'].to_csv(r'i86-ip.txt', header=False, index=None, sep=' ')

In [143]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/node-exporter-collectors
    export DOCKER_HOST="tcp://$ip:2376"
    docker stop nodeexporter  2>/dev/null
    docker stop monitor_tempsensor_1  2>/dev/null
    docker rm nodeexporter     2>/dev/null
    docker rm monitor_tempsensor_1  2>/dev/null
    docker-compose -f docker-compose-x86.yaml down 2>/dev/null
    docker-compose -f docker-compose-x86.yaml up -d 2>/dev/null
done <i86-ip.txt
cd $HOME_DIR

/root/04-deploy-monitor
nodeexporter
nodeexporter
nodeexporter
nodeexporter
nodeexporter
nodeexporter
nodeexporter
nodeexporter


### Deploy Tegra(nvidia jetson)

In [144]:
hosts[(hosts.arch == "aarch64")]['ip'].to_csv(r'aarch64-ip.txt', header=False, index=None, sep=' ')

In [145]:
%%bash
export HOME_DIR=$PWD
echo $HOME_DIR
source ../common/env.sh 
while read ip; do
    cd ~/repos/node-exporter-collectors
    export DOCKER_HOST="tcp://$ip:2376"
    docker stop nodeexporter  2>/dev/null
    docker rm nodeexporter   2>/dev/null  
    docker-compose -f docker-compose-jetson.yaml down 2>/dev/null
    docker-compose -f docker-compose-jetson.yaml up -d 2>/dev/null
done <aarch64-ip.txt
cd $HOME_DIR

/root/04-deploy-monitor
nodeexporter
nodeexporter


### Validade Deploy

- `docker ps` all nodes
- `curl` all nodes

In [146]:
%%bash
echo "ip|id|image|ports|runnning" > containers.csv
source ../common/env.sh 
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker ps --filter "name=nodeexporter"  --format "$ip|{{.ID}}|{{.Image}}|{{.Ports}}|{{.RunningFor}}"  >> containers.csv 2>/dev/null || :
done <ip.txt    

In [147]:
containers = pd.read_csv("containers.csv", sep = '|')
pd.merge(containers, hosts, on='ip', how='inner').drop(columns=supressed_columns)

,id,image,ports,runnning,hostname,arch
0,fb1114ef5ff5,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,About a minute ago,pi-node2,armv7l
1,fbdf7410748b,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,About a minute ago,pi-node3,armv7l
2,c5c1b27b0828,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,34 seconds ago,kx-Lenovo-H520g,x86_64
3,bfb8d0b8a9fa,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,29 seconds ago,nuc-01,x86_64
4,b2424bf443eb,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,24 seconds ago,nuc-02,x86_64
5,f941a7dc93e0,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,19 seconds ago,nuc-03,x86_64
6,5dc6b94328d9,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,About a minute ago,pi-node4,armv7l
7,ce4abe4927a6,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,About a minute ago,pi-node5,armv7l
8,f94621b76cf6,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,58 seconds ago,pi-node7,armv7l
9,e9fbb0da5d92,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,50 seconds ago,pi-node8,armv7l


In [148]:
%%bash
echo "ip,result" > node-exporters.csv
while read ip; do
    echo "$ip,`curl --connect-timeout 10 -o /dev/null -s -w "%{http_code}\n" http://$ip:9100`"  >> node-exporters.csv
done <ip.txt   

In [149]:
containers = pd.read_csv("node-exporters.csv", sep = ',')
pd.merge(containers, hosts, on='ip', how='inner').drop(columns=supressed_columns)

,result,hostname,arch
0,0,pi-node1,unknown
1,200,pi-node2,armv7l
2,200,pi-node3,armv7l
3,200,kx-Lenovo-H520g,x86_64
4,200,nuc-01,x86_64
5,200,nuc-02,x86_64
6,200,nuc-03,x86_64
7,200,pi-node4,armv7l
8,200,pi-node5,armv7l
9,0,pi-node6,armv7l


Observe that some hosts are not `200`, because this are not active servers at this tests. 